# Checking enviroment states and action sets

In [1]:
import sys
import os.path
sys.path.append("..")
from SAC_Agent import *
from ENV_DETAILS import *

2023-09-16 10:33:36.051012: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Num devices available:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
if os.name == 'nt':
    main_hyper_dir = "D:\\Artificial_Intelligence\\Portfolio\\RL_updated\\Pendulum\\" # Windows
    conda_python_exec = 'C:\\Users\\yanie\\anaconda3\\envs\\ai_dev\\python.exe '# Windows
    logs_dir = main_hyper_dir+'Hyperparam_tuning\\' # Windows
else:
    main_hyper_dir = "/media/n/NewDisk/Artificial_Intelligence/Portfolio/RL_updated/Pendulum/" # Linux
    conda_python_exec = '/home/n/anaconda3/envs/ai_dev/bin/python '# Linux
    logs_dir = main_hyper_dir+'Hyperparam_tuning_a3c/' # Linux

ENV = "Pendulum-v1"
SUCESS_CRITERIA_VALUE = ENV_DETAILS[ENV]["SUCESS_CRITERIA_VALUE"]
SUCESS_CRITERIA_EPOCH = ENV_DETAILS[ENV]["SUCESS_CRITERIA_EPOCH"]
EPISODES = ENV_DETAILS[ENV]["EPISODES"]

In [3]:

env = gym.make(ENV)
env


<TimeLimit<OrderEnforcing<PassiveEnvChecker<PendulumEnv<Pendulum-v1>>>>>

In [4]:
s = env.observation_space.sample()
s

array([0.36121026, 0.36347365, 6.0596123 ], dtype=float32)

In [5]:
env.observation_space, env.observation_space.shape

(Box([-1. -1. -8.], [1. 1. 8.], (3,), float32), (3,))

In [6]:
s = env.reset()
s[0]

array([ 0.06858359, -0.9976454 , -0.15465626], dtype=float32)

In [7]:
s = env.reset()[0]
env.action_space, env.action_space, env.step(s)

(Box(-2.0, 2.0, (1,), float32),
 Box(-2.0, 2.0, (1,), float32),
 (array([ 0.9849882 ,  0.17262147, -0.5705167 ], dtype=float32),
  -0.11710407005171813,
  False,
  False,
  {}))

### Training/Hyperparam run

In [8]:
%tensorboard --logdir logs_general/hyper

UsageError: Line magic function `%tensorboard` not found.


In [9]:
TUNING_TYPE = "BAYES"
HYPERPARAM_TUNING = True
writer= "Training/fit_A3C/"

In [10]:
if HYPERPARAM_TUNING:

    dir = r"Hyperparam_kt_sac"
    project_name = "keras_tunning_soft"

    tuner = kt.BayesianOptimization(
            MyHyperModel( hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/sac/",
                        end_of_episode = EPISODES, evaluation_epoch = env._max_episode_steps, training_steps = 600000,
                sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH, sucess_criteria_value= SUCESS_CRITERIA_VALUE,
                discount_min = 0.98, discount_max = 0.99,
                lr_actor_min = 0.000001, lr_actor_max = 0.001,
                lr_critic_1_min = 0.000001, lr_critic_1_max = 0.001,
                dense_min = 32, dense_max = 256,
                environment_name=ENV,
                tau_min = 0.001, tau_max = 0.1,
                reward_scaler = 16.2736044, num_layers_act = 3, num_layers_crit =3),
            objective= kt.Objective('total_train_reward', direction="max"), 
            max_trials = 12,
            directory=dir,
            project_name=project_name
        )
    tuner.search(x=[0], y=[1])
else : 
    
        print("Acquiring parameters ....")
        writer= "Training/fit_sac/"

        training_steps = 1000000
        learning_rate= 0.0001
        entropy_factor = 0.1
        discount = 0.99
        dense_units_actor = [128,64]
        num_layers_actor = 2
        dense_units_critic = [128,64]
        num_layers_crit = 2
        time_to_update= 100
        end_of_episode = 300
        tau = 0.01
        
        run_training(training_steps,  discount, dense_units_actor,  dense_units_critic, num_layers_actor, num_layers_crit, writer, end_of_episode,
                      environment_name = ENV,reward_scaler = 10, return_agent = False,lr_actor= 0.001, lr_critic_1= 0.001, lr_alpha = 0.001, tau = 0.001)
        

Trial 12 Complete [06h 58m 40s]
total_train_reward: -157.4864596443088

Best total_train_reward So Far: -149.4318500352101
Total elapsed time: 17h 42m 47s
INFO:tensorflow:Oracle triggered exit


In [11]:
#Trial id :04 | Score :-149.4318500352101 --> {'lr_actor': 0.0009451471031759065, 'lr_critic_1': 0.0006964871682796604, 'lr_alpha': 0.00031190366852756755, 'tau': 0.03118573469639496, 'n_dense_layers_actor': 3, 'dense_units_act_0': 120, 'dense_units_act_1': 203, 'dense_units_act_2': 103, 'n_dense_layers_critic': 2, 'dense_units_crit_0': 192, 'dense_units_crit_1': 78, 'dense_units_crit_2': 153}

exploration_tech = "soft"
hyperparam_combination=[]
for trials in tuner.oracle.get_best_trials(num_trials=3):
    print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)

Trial id :04 | Score :-149.4318500352101 --> {'lr_actor': 0.0009451471031759065, 'lr_critic_1': 0.0006964871682796604, 'lr_alpha': 0.00031190366852756755, 'tau': 0.03118573469639496, 'n_dense_layers_actor': 3, 'dense_units_act_0': 120, 'dense_units_act_1': 203, 'dense_units_act_2': 103, 'n_dense_layers_critic': 2, 'dense_units_crit_0': 192, 'dense_units_crit_1': 78, 'dense_units_crit_2': 153}
Trial id :03 | Score :-149.55966165184162 --> {'lr_actor': 0.0007609819189817605, 'lr_critic_1': 0.0005929755130071935, 'lr_alpha': 0.0007365211940950502, 'tau': 0.05267715818287114, 'n_dense_layers_actor': 3, 'dense_units_act_0': 243, 'dense_units_act_1': 211, 'dense_units_act_2': 64, 'n_dense_layers_critic': 1, 'dense_units_crit_0': 171, 'dense_units_crit_1': 49, 'dense_units_crit_2': 66}
Trial id :01 | Score :-152.349478138558 --> {'lr_actor': 0.000744024924747918, 'lr_critic_1': 0.0007270622789470238, 'lr_alpha': 0.0002715529104948473, 'tau': 0.030360207684080768, 'n_dense_layers_actor': 2, 'd

In [12]:

val_env = gym.make(ENV, render_mode = "rgb_array")
dir = r"Hyperparam_kt_sac"

for trials in tuner.oracle.get_best_trials(num_trials=1):
    print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)
    
env_model = tuner.get_best_models()[0]
final_rewards = final_evaluation(env_model,val_env,n_tries=200, exploration=exploration_tech,  video_name = "./sac_"+exploration_tech+"_video.mp4")
print("Final mean reward '",exploration_tech,"':", np.mean(final_rewards))

Trial id :04 | Score :-149.4318500352101 --> {'lr_actor': 0.0009451471031759065, 'lr_critic_1': 0.0006964871682796604, 'lr_alpha': 0.00031190366852756755, 'tau': 0.03118573469639496, 'n_dense_layers_actor': 3, 'dense_units_act_0': 120, 'dense_units_act_1': 203, 'dense_units_act_2': 103, 'n_dense_layers_critic': 2, 'dense_units_crit_0': 192, 'dense_units_crit_1': 78, 'dense_units_crit_2': 153}
Trial number :  12
Moviepy - Building video ./sac_soft_video.mp4.
Moviepy - Writing video ./sac_soft_video.mp4



Moviepy - Done !
Moviepy - video ready ./sac_soft_video.mp4
Final mean reward ' soft ': -139.38614166591353
